# Random Forest

## Libraries

In [1]:
# working with data
import numpy as np
import pandas as pd
# data visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors
# modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectFromModel
# saving model
import pickle

## Data Preparation

In [2]:
# Read in dataset from csv
songs = pd.read_csv("songs.csv", index_col=False)
df_no_genres = songs.drop(columns=['track_id', 'track_name', 'genres'])
df_genres = songs.drop(columns=['track_id', 'track_name'])

In [3]:
# Split dataset into train and test
y = df_no_genres['user_like']
X = df_no_genres.drop(columns='user_like')
X, y = shuffle(X, y, random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [4]:
# Verify data shape after split
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(384, 14)
(384,)
(97, 14)
(97,)


## Model Building

In [5]:
# Defining evaluation criteria
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = np.mean(errors) * 100
    accuracy = 100 - mape
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

# Using default settings
base_model = RandomForestClassifier(n_estimators = 100, random_state = 123)

Starting from the simpliest random forest model using default parameters, I want to use cross validation to see the optimal number of features to use. The idea is to use cross validation to calculate an average error and see if we can reduce the error by removing the least important feature iteratively.

In [6]:
def get_most_important_features(X, y):
    model = RandomForestClassifier(n_estimators = 100, random_state = 123)
    # initialize cv score with base model
    base_cv_score = np.mean(cross_val_score(model, X, y, cv=10)) 
    max_cv_score = base_cv_score
    cur_features, new_features = X.columns, X.columns
    new_train = X
    while True:
        # remove feature that results in least impurity decrease
        sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state = 123), max_features=len(new_features)-1, threshold=-np.inf)
        sel.fit(new_train, y)
        new_features = cur_features[sel.get_support()]
        new_train = X[new_features]
        
        # cv score with reduced model
        new_cv_score = np.mean(cross_val_score(model, new_train, y, cv=10))
        
        # update max cv score if model improves
        if new_cv_score > max_cv_score:
            max_cv_score = new_cv_score
            cur_features = new_features
        else:
            break
    print(f"Base CV score: {base_cv_score}")
    print(f"Final CV score: {max_cv_score}")
    print(f"Features to use: {cur_features}")
    return cur_features

reduced_features = get_most_important_features(X_train, y_train)

Base CV score: 0.6980431848852902
Final CV score: 0.7139676113360325
Features to use: Index(['popularity', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms'],
      dtype='object')


Using 10-fold cross validation it looks like we can omit `time_signature` and `mode`. 

In [7]:
X_train = X_train[reduced_features]
X_test = X_test[reduced_features]

In [8]:
base_model.fit(X_train, y_train)
evaluate(base_model, X_test, y_test)

Average Error: 0.2268 degrees.
Accuracy = 77.32%.


Using the random forest model with default parameters and omitting `time_signature` and `mode`, we obtain an evaluation accuracy of 77.32%. There's definitely room for improvement. The next step will be to tune the model's hyperparameters.

In [9]:
base_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 123,
 'verbose': 0,
 'warm_start': False}

There are quite a few hyperparameters for our model... Judging from prior experience however, the most impactful ones tend to be:
- the number of decision trees in the forest (i.e. `n_estimators`)
- the maximum number of features to consider when splitting a node (i.e. `max_features`)
- the minimum number of data points allowed in a leaf node (i.e. `min_samples_leaf`)

As a starting point, we can perform random grid search on ideal hyperparameter values. With a general idea of the ideal hyperparameter values we can then refine it with a narrower grid search.

In [10]:
# grid of hyperparamter values to test
# rand_grid_values = {'max_features': ['sqrt', 'log2', None, 0.5, 0.7, 0.9],
#  'min_samples_leaf': [2, 4, 8, 16],
#  'max_depth': [3, 5, 7],
#  'n_estimators': [100, 200, 400, 800]}
rand_grid_values = {'n_estimators':np.arange(100, 200, 400),
              'max_features':np.arange(0.1, 1, 0.1),
              'max_depth': [3, 5, 7, 9],
              'max_samples': [0.3, 0.5, 0.8]}

In [11]:
model_random = RandomizedSearchCV(estimator = base_model, 
                                  param_distributions = rand_grid_values,
                                  n_iter = 100,
                                  cv = 3,
                                  random_state = 123,
                                  n_jobs = -1)

In [12]:
model_random.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=123),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 7, 9],
                                        'max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'max_samples': [0.3, 0.5, 0.8],
                                        'n_estimators': array([100])},
                   random_state=123)

In [13]:
model_random.best_params_

{'n_estimators': 100, 'max_samples': 0.5, 'max_features': 0.8, 'max_depth': 7}

In [14]:
evaluate(model_random, X_test, y_test)

Average Error: 0.2474 degrees.
Accuracy = 75.26%.


An improvement of ~ 1% in accuracy. And now for a more refined grid search.

In [15]:
# grid_values = {'max_features': [0.6, 0.7, 0.8],
#  'min_samples_leaf': [2, 3],
#  'n_estimators': [400]}
grid_values = {'max_features': [0.8],
 'max_depth': [7],
 'max_samples': [0.4, 0.5, 0.6],
 'n_estimators': [400]}

model_gridsearch = GridSearchCV(estimator = base_model, 
                                param_grid = grid_values,                                  
                                cv = 10,                                 
                                n_jobs = -1)

In [16]:
model_gridsearch.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=123),
             n_jobs=-1,
             param_grid={'max_depth': [7], 'max_features': [0.8],
                         'max_samples': [0.4, 0.5, 0.6],
                         'n_estimators': [400]})

In [17]:
model_gridsearch.best_params_

{'max_depth': 7, 'max_features': 0.8, 'max_samples': 0.4, 'n_estimators': 400}

In [18]:
evaluate(model_gridsearch, X_test, y_test)

Average Error: 0.2371 degrees.
Accuracy = 76.29%.


It looks like we can't improve the hyperparameters any further with the narrower grid search. It's important to realize what these hyperparameter values suggest. If I allowed `min_sample_lead` to be 1, this would mean a leaf node can contain only 1 sample. This allows the possibility of really deep decision trees. Similarly, if I allowed `max_features` = 1, this would mean each tree in the random forest only considers one feature at a time when splitting a node. The complications of these two hyperparameter values (if allowed to be 1) is that we risk overfitting. 

In [19]:
final_model = RandomForestClassifier(max_features=0.8, 
                                    max_depth=7,
                                    n_estimators=400,
                                    max_samples=0.3,
                                    random_state=123)
final_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, max_features=0.8, max_samples=0.3,
                       n_estimators=400, random_state=123)

In [20]:
evaluate(final_model, X_test, y_test)

Average Error: 0.1959 degrees.
Accuracy = 80.41%.


In [24]:
# fit on whole dataset
final_model.fit(X.drop(columns=['mode', 'time_signature']), y)
# write model to disk
pickle.dump(final_model, open('rf.sav', 'wb'))